In [18]:
# Import machine learning packages for ease
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.feature_selection import VarianceThreshold, RFE, SequentialFeatureSelector
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
from scipy.spatial import distance, distance_matrix
from difflib import SequenceMatcher
import warnings
import time
from tqdm import tqdm

In [19]:
# Suppress warnings that are typically produced during automated ML tasks
warnings.filterwarnings('ignore')

In [20]:
# Load the state-wise accident dataset
US_dataset = pd.read_csv('States.csv', sep=',')
print(US_dataset.head)

# Extract the column names and present them
US_names = US_dataset.columns.to_list()
print(US_names)

# Show number of NaNs
print(f'# NaNs before drop: {US_dataset.isna().sum().sum()}')

# Interpolate to rid of NaNs
US_dataset = US_dataset.dropna()

# Show number of NaNs
print(f'# NaNs after drop: {US_dataset.isna().sum().sum()}')

<bound method NDFrame.head of                 ID  Severity           Start_Time             End_Time   
0              A-1         3  2016-02-08 00:37:08  2016-02-08 06:37:08  \
1              A-2         2  2016-02-08 05:56:20  2016-02-08 11:56:20   
2              A-3         2  2016-02-08 06:15:39  2016-02-08 12:15:39   
3              A-4         2  2016-02-08 06:51:45  2016-02-08 12:51:45   
4              A-5         3  2016-02-08 07:53:43  2016-02-08 13:53:43   
...            ...       ...                  ...                  ...   
2845337  A-2845338         2  2019-08-23 18:03:25  2019-08-23 18:32:01   
2845338  A-2845339         2  2019-08-23 19:11:30  2019-08-23 19:38:23   
2845339  A-2845340         2  2019-08-23 19:00:21  2019-08-23 19:28:49   
2845340  A-2845341         2  2019-08-23 19:00:21  2019-08-23 19:29:42   
2845341  A-2845342         2  2019-08-23 18:52:06  2019-08-23 19:21:31   

         Start_Lat   Start_Lng    End_Lat     End_Lng  Distance(mi)   
0        4

In [21]:
# Splits of dataset by State
splits = US_dataset.groupby('State')

# Create dictionary of dataframes for states
US_States_datasets = {k: v for k, v in splits}

# Present states and their data shapes
for k, v in US_States_datasets.items():
    print(f'{k}: {v.shape}')

AL: (5093, 47)
AR: (2889, 47)
AZ: (20138, 47)
CA: (215629, 47)
CO: (3570, 47)
CT: (2571, 47)
DC: (3385, 47)
DE: (1474, 47)
FL: (217566, 47)
GA: (2858, 47)
IA: (1318, 47)
ID: (3396, 47)
IL: (9853, 47)
IN: (2229, 47)
KS: (359, 47)
KY: (1078, 47)
LA: (25843, 47)
MA: (546, 47)
MD: (11635, 47)
ME: (167, 47)
MI: (10952, 47)
MN: (26337, 47)
MO: (4184, 47)
MS: (823, 47)
MT: (7895, 47)
NC: (42677, 47)
ND: (448, 47)
NE: (568, 47)
NH: (218, 47)
NJ: (7675, 47)
NM: (162, 47)
NV: (788, 47)
NY: (23138, 47)
OH: (6444, 47)
OK: (5041, 47)
OR: (45295, 47)
PA: (48470, 47)
RI: (92, 47)
SC: (57481, 47)
SD: (122, 47)
TN: (21480, 47)
TX: (52073, 47)
UT: (10843, 47)
VA: (30320, 47)
VT: (102, 47)
WA: (6418, 47)
WI: (496, 47)
WV: (1134, 47)
WY: (45, 47)


In [22]:
# Clean up state datasets
for US_State_name, US_State_dataset in tqdm(US_States_datasets.items()):

    # Removes state column since completely unecessary now
    US_State_dataset.drop(columns=['State'])

    # Performs encoding to remove string-type data
    for col in US_State_dataset.columns:
        if US_State_dataset[col].dtype == 'object':
            le = LabelEncoder()
            le.fit(US_State_dataset[col])
            US_State_dataset[col] = le.transform(US_State_dataset[col])

    # To stick with budget on time for assignment, reduce any datasets with over 5000 samples to 5000 samples
    if US_State_dataset.shape[0] > 5000:
        US_States_datasets[US_State_name] = resample(US_State_dataset, n_samples=5000)

100%|██████████| 49/49 [00:08<00:00,  5.95it/s]


# EXPERIMENT

- [ ] For each feature selection method
  - [x] Time for computation?
  - [ ] What level of variance was there between each state in terms of features selected?
  - [x] Calculate the mean performance after fitting and cross validating on a choice ML algorithm for each US state subdataset (see **'$'** below)
- [x] For each US State subdataset in feature selection method
  - [x] What top 10 features were selection?
    - [x] Is there an order to them?
  - [x] **$** Using selected features, fit and cross validate on a choice ML algorithm
    - [x] What was the performance?

## How to measure distance or variance between lists

1. Combine all lists into a 2D array  
**data = np.array([list1, list2, list3])**

2. Calculate the pairwise Euclidean distance between lists  
**distances = distance.pdist(data, 'hamming')**

3. Convert the pairwise distances into a matrix  
**matrix = distance_matrix(data, data)**

4. Calculate the average distance between lists  
**average_distance = np.mean(matrix)**

5. Print results  
**print("Average distance:", average_distance)**

In [23]:
"""
1. Sequential Feature Selection
"""

# Collect start time
start = time.time()

# List for storing scores
scores = []

# List for storing features selected
all_features = []

# Perform experiment over each state
for US_State_name, US_State_dataset in US_States_datasets.items():

    # separate the features and target
    X = US_State_dataset.drop(['Severity'], axis=1)
    y = US_State_dataset['Severity']

    # Create a machine learning algorithm
    model = LogisticRegression()

    # Perform RFE on state data
    sfs = SequentialFeatureSelector(model, n_features_to_select=10, n_jobs=-1)

    # Fit using the RFE method
    sfs.fit(X, y)

    # Add to features list
    features = X.columns[sfs.support_].tolist()
    all_features.append(features)

    # Print best features for state
    print(f'{US_State_name}: {features}', end='')

    # Obtain new X
    X_new = sfs.transform(X)

    # Create and fit model for predictions using selected features
    fitted_model = MLPClassifier()

    # Perform cross validation
    score = cross_val_score(fitted_model, X_new, y, cv=3, scoring='accuracy').mean()

    # Present score for this state
    print(f' -- score: {score}')

    # Store score
    scores.append(score)

# Present final score
print(f'mean score: {sum(scores) / len(scores)}')

# Calculate and present feature variance 
total_difference = 0
for i in range(len(all_features)):
    for j in range(len(all_features)):
        if i != j:
            difference = SequenceMatcher(None, all_features[i], all_features[j]).ratio()
            total_difference += difference
average_difference = total_difference / ((len(all_features) * (len(all_features) - 1)) / 2)
print("Average difference:", average_difference)

# Present time elapsed
end = time.time()
print(f'time elapsed: {end - start}')

AL: ['ID', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number'] -- score: 0.8318037472937586
AR: ['ID', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Number', 'Street', 'Side', 'City'] -- score: 0.570785739010038
AZ: ['Distance(mi)', 'Description', 'State', 'Country', 'Timezone', 'Weather_Timestamp', 'Railway', 'Roundabout', 'Stop', 'Sunrise_Sunset'] -- score: 0.8385962663409695
CA: ['ID', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Number', 'Side'] -- score: 0.9650012374475884
CO: ['Start_Time', 'Description', 'County', 'State', 'Zipcode', 'Country', 'Airport_Code', 'Amenity', 'Bump', 'Sunrise_Sunset'] -- score: 0.5957983193277311
CT: ['Start_Lat', 'End_Lat', 'Description', 'State', 'Country', 'Timezone', 'Wind_Chill(F)', 'Junction', 'Station', 'Astronomical_Twilight'] -- score: 0.6666666666666666
DC: ['ID', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Di

In [24]:
"""
2. Recursive Feature Elimination
"""

# Collect start time
start = time.time()

# List for storing scores
scores = []

# List for features selected
all_features = []

# Perform experiment over each state
for US_State_name, US_State_dataset in US_States_datasets.items():

    # separate the features and target
    X = US_State_dataset.drop(['Severity'], axis=1)
    y = US_State_dataset['Severity']

    # Create a machine learning algorithm
    model = LogisticRegression()

    # Perform RFE on state data
    rfe = RFE(model, n_features_to_select=10)

    # Fit using the RFE method
    rfe.fit(X, y)

    # Add to features list
    features = X.columns[rfe.support_].tolist()
    all_features.append(features)

    # Print best features for state
    print(f'{US_State_name}: {features}', end='')

    # Obtain new X
    X_new = rfe.transform(X)

    # Create and fit model for predictions using selected features
    fitted_model = MLPClassifier()

    # Perform cross validation
    score = cross_val_score(fitted_model, X_new, y, cv=3, scoring='accuracy').mean()

    # Present score for this state
    print(f' -- score: {score}')

    # Store score
    scores.append(score)

# Present final score
print(f'mean score: {sum(scores) / len(scores)}')

# Calculate and present feature variance 
total_difference = 0
for i in range(len(all_features)):
    for j in range(len(all_features)):
        if i != j:
            difference = SequenceMatcher(None, all_features[i], all_features[j]).ratio()
            total_difference += difference
average_difference = total_difference / ((len(all_features) * (len(all_features) - 1)) / 2)
print("Average difference:", average_difference)

# Present time elapsed
end = time.time()
print(f'time elapsed: {end - start}')

AL: ['End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'County', 'Airport_Code', 'Weather_Timestamp', 'Humidity(%)', 'Wind_Direction'] -- score: 0.911200472990636
AR: ['Start_Time', 'End_Time', 'Start_Lng', 'End_Lng', 'Description', 'County', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)'] -- score: 0.6580131533402561
AZ: ['Start_Time', 'End_Time', 'Start_Lng', 'End_Lat', 'End_Lng', 'Description', 'City', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)'] -- score: 0.9602002768713964
CA: ['Start_Time', 'End_Time', 'Start_Lng', 'End_Lng', 'Description', 'City', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)'] -- score: 0.9653943160947641
CO: ['Start_Time', 'End_Time', 'Start_Lng', 'End_Lng', 'County', 'Airport_Code', 'Weather_Timestamp', 'Wind_Chill(F)', 'Humidity(%)', 'Weather_Condition'] -- score: 0.3417366946778711
CT: ['Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'Description', 'County', 'Airport_Code', 'W

In [25]:
"""
3. Variance Thresholding
"""

# Collect start time
start = time.time()

# List for storing scores
scores = []

# List for storing features selected
all_features = []

# Perform experiment over each state
for US_State_name, US_State_dataset in US_States_datasets.items():

    # separate the features and target
    X = US_State_dataset.drop(['Severity'], axis=1)
    y = US_State_dataset['Severity']

    # Perform variance threshold on X
    var = VarianceThreshold(0.01)

    # Create new X with variance threshold applied
    X_new = var.fit_transform(X)

    # Find the 10 best features
    top10 = var.get_support(indices=True)[:10]

    # Add to features list
    features = X.columns[top10].to_list()
    all_features.append(features)

    # Present 10 best features
    print(f'{US_State_name}: {features}', end='')

    # Create and fit model for predictions using selected features
    fitted_model = MLPClassifier()

    # Perform cross validation
    score = cross_val_score(fitted_model, X_new, y, cv=3, scoring='accuracy').mean()

    # Present score for this state
    print(f' -- score: {score}')

    # Store score
    scores.append(score)

# Present final score
print(f'mean score: {sum(scores) / len(scores)}')

# Calculate and present feature variance 
total_difference = 0
for i in range(len(all_features)):
    for j in range(len(all_features)):
        if i != j:
            difference = SequenceMatcher(None, all_features[i], all_features[j]).ratio()
            total_difference += difference
average_difference = total_difference / ((len(all_features) * (len(all_features) - 1)) / 2)
print("Average difference:", average_difference)

# Present time elapsed
end = time.time()
print(f'\ttime elapsed: {end - start}')

AL: ['ID', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number'] -- score: 0.8630073505106902
AR: ['ID', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number'] -- score: 0.6992038767739702
AZ: ['ID', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number'] -- score: 0.9485976754228987
CA: ['ID', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number'] -- score: 0.9753989178154766
CO: ['ID', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number'] -- score: 0.3568627450980392
CT: ['ID', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number'] -- score: 0.5285880980163361
DC: ['ID', 'Start_Time', 'End_Time', 'Distance(mi)', 'Description', 'N

In [26]:
"""
4. Random Forest-based Feature Selection
"""

# Collect start time
start = time.time()

# List for storing scores
scores = []

# List for features selected
all_features = []

# Perform experiment over each state
for US_State_name, US_State_dataset in US_States_datasets.items():

    # separate the features and target
    X = US_State_dataset.drop(['Severity'], axis=1)
    y = US_State_dataset['Severity']

    # Create random forest classifier
    rfc = RandomForestClassifier(n_estimators=500, random_state=42)

    # Fit random forest classifier to data
    rfc.fit(X, y)

    # Find the 10 best features
    top10 = pd.Series(rfc.feature_importances_, index=X.columns)

    # Sort the 10 best features by score (descending)
    top10sorted = top10.sort_values(ascending=False)

    # Add to features list
    features = top10sorted.index[:10].to_list()
    all_features.append(features)

    # Present the top 10 features
    print(f'{US_State_name}: {features}', end='')

    # Create a new X containing only the selected features
    X_new = X[top10sorted.index.tolist()]

    # Create and fit model for predictions using selected features
    fitted_model = MLPClassifier()

    # Perform cross validation
    score = cross_val_score(fitted_model, X_new, y, cv=3, scoring='accuracy').mean()

    # Present score for this state
    print(f' -- score: {score}')

    # Store score
    scores.append(score)

# Present final score
print(f'mean score: {sum(scores) / len(scores)}')

# Calculate and present feature variance 
total_difference = 0
for i in range(len(all_features)):
    for j in range(len(all_features)):
        if i != j:
            difference = SequenceMatcher(None, all_features[i], all_features[j]).ratio()
            total_difference += difference
average_difference = total_difference / ((len(all_features) * (len(all_features) - 1)) / 2)
print("Average difference:", average_difference)

# Present time elapsed
end = time.time()
print(f'time elapsed: {end - start}')

AL: ['Description', 'End_Time', 'ID', 'Start_Time', 'End_Lng', 'Weather_Timestamp', 'Start_Lng', 'Zipcode', 'Start_Lat', 'End_Lat'] -- score: 0.8588056698384213
AR: ['Description', 'End_Time', 'Start_Time', 'ID', 'Weather_Timestamp', 'Distance(mi)', 'End_Lat', 'Start_Lat', 'Zipcode', 'End_Lng'] -- score: 0.7282796815507097
AZ: ['Distance(mi)', 'Description', 'Weather_Timestamp', 'End_Time', 'Start_Time', 'ID', 'Traffic_Signal', 'Crossing', 'Humidity(%)', 'Temperature(F)'] -- score: 0.9463972751668154
CA: ['Description', 'Distance(mi)', 'Start_Time', 'End_Time', 'Weather_Timestamp', 'ID', 'Start_Lng', 'End_Lng', 'Humidity(%)', 'Start_Lat'] -- score: 0.9756007982077053
CO: ['Description', 'Start_Time', 'End_Time', 'Weather_Timestamp', 'ID', 'Distance(mi)', 'Zipcode', 'End_Lat', 'Start_Lat', 'Start_Lng'] -- score: 0.3887955182072829
CT: ['Description', 'End_Time', 'Start_Time', 'Weather_Timestamp', 'ID', 'End_Lat', 'Start_Lat', 'Zipcode', 'End_Lng', 'Distance(mi)'] -- score: 0.57215091404

In [27]:
"""
5. Neural Network-based Feature Selection
"""

# Collect start time
start = time.time()

# List for storing scores
scores = []

# List for features selected
all_features = []

# Perform experiment over each state
for US_State_name, US_State_dataset in US_States_datasets.items():

    # separate the features and target
    X = US_State_dataset.drop(['Severity'], axis=1)
    y = US_State_dataset['Severity']

    # Create MLP Classifier
    mlpc = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000)

    # Fit MLP Classifier to data
    mlpc.fit(X, y)

    # use permutation feature importance to rank the features
    top10 = permutation_importance(mlpc, X, y, n_repeats=10)

    # sort the feature importance scores in descending order
    top10sorted = top10.importances_mean.argsort()[::-1]

    # Add to features list
    features = [X.columns[top10sorted[i]] for i in range(10)]
    all_features.append(features)

    # Present the top 10 features
    print(f'{US_State_name}: {features}', end='')

    # Get the list of feature names from top10sorted
    feature_names = X.columns[top10sorted].tolist()

    # Create a new X containing only the selected features
    X_new = X[feature_names]

    # Create and fit model for predictions using selected features
    fitted_model = MLPClassifier()

    # Perform cross validation
    score = cross_val_score(fitted_model, X_new, y, cv=3, scoring='accuracy').mean()

    # Present score for this state
    print(f' -- score: {score}')

    # Store score
    scores.append(score)

# Present final score
print(f'mean score: {sum(scores) / len(scores)}')

# Calculate and present feature variance 
total_difference = 0
for i in range(len(all_features)):
    for j in range(len(all_features)):
        if i != j:
            difference = SequenceMatcher(None, all_features[i], all_features[j]).ratio()
            total_difference += difference
average_difference = total_difference / ((len(all_features) * (len(all_features) - 1)) / 2)
print("Average difference:", average_difference)

# Present time elapsed
end = time.time()
print(f'time elapsed: {end - start}')

AL: ['Weather_Timestamp', 'End_Time', 'Start_Time', 'ID', 'Description', 'Number', 'Street', 'Zipcode', 'Airport_Code', 'Humidity(%)'] -- score: 0.8736036706224182
AR: ['Start_Time', 'Weather_Timestamp', 'End_Time', 'Description', 'County', 'ID', 'Zipcode', 'Wind_Chill(F)', 'Number', 'Street'] -- score: 0.6767047421253028
AZ: ['End_Time', 'Start_Time', 'ID', 'Weather_Timestamp', 'Description', 'Number', 'Street', 'Zipcode', 'End_Lng', 'Distance(mi)'] -- score: 0.9445954746625705
CA: ['Start_Time', 'Weather_Timestamp', 'End_Time', 'ID', 'Description', 'State', 'Pressure(in)', 'Nautical_Twilight', 'Wind_Chill(F)', 'Timezone'] -- score: 0.9733965571831611
CO: ['Weather_Timestamp', 'Start_Time', 'Description', 'End_Time', 'Number', 'Zipcode', 'Street', 'County', 'City', 'ID'] -- score: 0.39327731092436974
CT: ['Weather_Timestamp', 'Start_Time', 'End_Time', 'ID', 'Description', 'Zipcode', 'Number', 'Street', 'Temperature(F)', 'Wind_Chill(F)'] -- score: 0.5425904317386231
DC: ['Weather_Times